In [1]:
import requests
import pandas as pd
%load_ext kedro.ipython

[12/02/24 13:19:23] INFO     Using                                                                  ]8;id=29353;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=264044;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/framework/project/__init__.py#270\270]8;;\
                             '/root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/framework/pr                
                             oject/rich_logging.yml' as logging configuration.                                     

                    INFO     Registered line magic '%reload_kedro'                                   ]8;id=348158;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=827789;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#61\61]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=497129;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=558413;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#63\63]8;;\

                    INFO     Resolved project path as: /root/kedro-cic.                             ]8;id=391964;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=319281;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#178\178]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[12/02/24 13:19:26] WARNING  /root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/framework/pro ]8;id=590505;file:///root/.pyenv/versions/3.10.12/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=174798;file:///root/.pyenv/versions/3.10.12/lib/python3.10/warnings.py#109\109]8;;\
                             ject/__init__.py:347: UserWarning: The                                                
                             'kedro_cic.pipelines.openalex_works' module does not expose a                         
                             'create_pipeline' function, so no pipelines defined therein will be                   
                             returned by 'find_pipelines'.                                                         
                               warnings.warn(                                                                      
                                                                                                                   

                    INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=674897;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=176783;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro_telemetry/plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[12/02/24 13:19:27] INFO     Kedro project kedro-cic                                                ]8;id=610831;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=756279;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#144\144]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=226801;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=655607;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#145\145]8;;\
                             'pipelines'                                                                           

[12/02/24 13:19:30] INFO     Registered line magic 'run_viz'                                        ]8;id=385270;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=621600;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#151\151]8;;\

In [2]:
institution_ror = catalog.load('params:openalex_fetch_options.institution_ror')
#env = catalog.load('params:fetch_options.env')
env = 'dev'

                    INFO     Loading data from params:openalex_fetch_options.institution_ror    ]8;id=158775;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=825785;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

In [3]:
env

'dev'

In [ ]:
def fetch_author_openalex(institution_ror, env):
    cursor = '*'
    base_url = 'https://api.openalex.org/authors?filter=affiliations.institution.ror:{}&cursor={}'
    iteration_limit = 5
    iteration_count = 0

    url = base_url.format(institution_ror, cursor)
    api_response = requests.get(url).json()

    print(f'Iteration count: {iteration_count}')
    print(f'GET {url}')

    # creo dataframe con las columnas del primer resultado 
    df = pd.DataFrame.from_dict(api_response['results'])

    # update cursor
    cursor = api_response['meta']['next_cursor']
    url = base_url.format(institution_ror, cursor)

    while cursor:

        if env == 'dev' and iteration_count >= iteration_limit:
            break

        # request api with updated cursor
        url = base_url.format(institution_ror, cursor)

        iteration_count += 1
        print(f'Iteration count: {iteration_count}')
        print(f'GET {url}')

        api_response = requests.get(url).json()

        df_tmp = pd.DataFrame.from_dict(api_response['results'])

        df = pd.concat([df, df_tmp])

        # update cursor
        cursor = api_response['meta']['next_cursor']

    return df

In [5]:
df = fetch_work_openalex(institution_ror, env)
df

Iteration count: 0
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/02s7sax82&cursor=*
Iteration count: 1
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/02s7sax82&cursor=IlsyMzUsICdodHRwczovL29wZW5hbGV4Lm9yZy9BNTEwMDY3NTM4NSddIg==
Iteration count: 2
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/02s7sax82&cursor=IlsxNzksICdodHRwczovL29wZW5hbGV4Lm9yZy9BNTA2MTYwMzM1NyddIg==
Iteration count: 3
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/02s7sax82&cursor=IlsxNTYsICdodHRwczovL29wZW5hbGV4Lm9yZy9BNTA0ODA2NjU3NiddIg==
Iteration count: 4
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/02s7sax82&cursor=IlsxNDIsICdodHRwczovL29wZW5hbGV4Lm9yZy9BNTAxNjA2MDc3NCddIg==
Iteration count: 5
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/02s7sax82&cursor=Ilsx

,id,orcid,display_name,display_name_alternatives,works_count,cited_by_count,summary_stats,ids,affiliations,last_known_institutions,topics,topic_share,x_concepts,counts_by_year,works_api_url,updated_date,created_date
0,https://openalex.org/A5003447605,https://orcid.org/0000-0002-5260-1807,Gustavo E. Romero,"[Gladis Graciela Romero, Gustavo Romero, Gusta...",760,9045,"{'2yr_mean_citedness': 1.2045454545454546, 'h_...",{'openalex': 'https://openalex.org/A5003447605...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I4210134895', 'r...","[{'id': 'https://openalex.org/T10818', 'displa...","[{'id': 'https://openalex.org/T10818', 'displa...","[{'id': 'https://openalex.org/C121332964', 'wi...","[{'year': 2024, 'works_count': 23, 'cited_by_c...",https://api.openalex.org/works?filter=author.i...,2024-11-27T02:59:56.541657,2023-07-21
1,https://openalex.org/A5053874532,None,Guillermo Soberón,"[Guillermo R Soberon, Guillermo Soberon, G. So...",629,1120,"{'2yr_mean_citedness': 0.0, 'h_index': 14, 'i1...",{'openalex': 'https://openalex.org/A5053874532'},[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I4210143748', 'r...","[{'id': 'https://openalex.org/T13464', 'displa...","[{'id': 'https://openalex.org/T13403', 'displa...","[{'id': 'https://openalex.org/C138885662', 'wi...","[{'year': 2024, 'works_count': 0, 'cited_by_co...",https://api.openalex.org/works?filter=author.i...,2024-11-27T21:11:57.848498,2023-07-21
2,https://openalex.org/A5112709549,None,Hugo Luis López,"[Hugo L. López, Hugo Luis López, Hugo L. Lopez...",480,3300,"{'2yr_mean_citedness': 0.3333333333333333, 'h_...",{'openalex': 'https://openalex.org/A5112709549'},[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I874386039', 'ro...","[{'id': 'https://openalex.org/T13589', 'displa...","[{'id': 'https://openalex.org/T13589', 'displa...","[{'id': 'https://openalex.org/C142362112', 'wi...","[{'year': 2024, 'works_count': 0, 'cited_by_co...",https://api.openalex.org/works?filter=author.i...,2024-11-22T12:03:52.544346,2024-10-15
3,https://openalex.org/A5019186384,https://orcid.org/0000-0001-6247-0428,Guillermo Ricardo Simari,"[Guillermo Ricardo Simari, G. Simari, Guillerm...",467,5900,"{'2yr_mean_citedness': 1.2857142857142858, 'h_...",{'openalex': 'https://openalex.org/A5019186384...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I56858762', 'ror...","[{'id': 'https://openalex.org/T10456', 'displa...","[{'id': 'https://openalex.org/T10456', 'displa...","[{'id': 'https://openalex.org/C138885662', 'wi...","[{'year': 2024, 'works_count': 1, 'cited_by_co...",https://api.openalex.org/works?filter=author.i...,2024-11-29T07:57:46.761250,2023-07-21
4,https://openalex.org/A5062756128,https://orcid.org/0000-0002-1030-4492,Peter Vogt,"[Peter Vogt, P.R Vogt, P. Vogt, V. Peter, P. R...",420,15770,"{'2yr_mean_citedness': 0.8888888888888888, 'h_...",{'openalex': 'https://openalex.org/A5062756128...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I4210118689', 'r...","[{'id': 'https://openalex.org/T10001', 'displa...","[{'id': 'https://openalex.org/T14323', 'displa...","[{'id': 'https://openalex.org/C86803240', 'wik...","[{'year': 2024, 'works_count': 4, 'cited_by_co...",https://api.openalex.org/works?filter=author.i...,2024-11-29T10:57:56.410709,2023-07-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,https://openalex.org/A5019105905,None,M. Forker,"[M. Forker, Manfred Forker]",118,1254,"{'2yr_mean_citedness': 0.0, 'h_index': 19, 'i1...",{'openalex': 'https://openalex.org/A5019105905'},[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I76248293', 'ror...","[{'id': 'https://openalex.org/T10681', 'displa...","[{'id': 'https://openalex.org/T10681', 'displa...","[{'id': 'https://openalex.org/C121332964', 'wi...","[{'year': 2024, 'works_count': 0, 'cited_by_co...",https://api.openalex